In [7]:
import torch
from torch import nn, optim
from torch.utils.data import (Dataset, DataLoader, TensorDataset)
import tqdm
from torchvision.datasets import FashionMNIST
from torchvision import transforms

#訓練用のデータを取得
#そのままだとPIL(Python Imaging Library)の画像形式で
#Datasetを作ってしまうので，
#transforms.ToTensorでTensorに変換する
fashion_mnist_train = FashionMNIST("~/Program_Practice/python/Pytorch/4/FashionMNIST", train=True, download=True, transform=transforms.ToTensor())
#検証用データの取得
fashion_mnist_test = FashionMNIST("~/Program_Practice/python/Pytorch/4/FashionMNIST", train=False, download=True, transform=transforms.ToTensor())

#バッチサイズが128のDataLoaderをそれぞれ作成
batch_size=128
train_loader = DataLoader(fashion_mnist_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(fashion_mnist_test, batch_size=batch_size, shuffle=False)

In [ ]:
#(N, C, H, W)形式のTensorを(N, C*H*W)に引き伸ばす層
#畳み込み層の出力をMLPに渡す際に必要
class FlattenLayer(nn.Module):
    def forward(self, x):
        sizes = x.size()
        return x.view(sizes[0], -1)

#5×5のカーネルを使用して，最初に32個，次に64個のチャンネルを作成する
#BatchNorm2dは画像形式用のBatch Normalization
#最後にFlattenLayerを挟む
conv_net = nn.Sequential(
    nn.Conv2d(1, 32, 5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.BatchNorm2d(32),
    nn.Dropout2d(0.25),
    nn.Conv2d(2),
    nn.ReLU(),
    nn.BatchNorm2d(64),
    nn.Dropout2d(0.25),
    FlattenLayer()
)

#畳み込みによって最終的にどのようなサイズになっているかを，
#実際にダミーデータを入れてみて確認する
text_input = torch.ones(1, 1, 28, 28)
conv_output_size = conv_net(test_input).size()[-1]

#2層のMLP
mlp = nn.Sequential(
    nn.Linear(conv_output_size, 200),
    nn.ReLU(),
    nn.BatchNormld(200),
    nn.Dropout(0.25),
    nn.Linear(200, 10)
)

#最終的なCNN
net = nn.Sequential(
    conv_net,
    mlp
)